In [ ]:
# 실습: 삼성전자 주가 데이터를 분석하는 어플리케이션을 function(tool)으로 수정하여, 다른 종목의 데이터를 정해진 내용대로 가져와서 답하는 어플리케이션을 작성하시오.

In [1]:
! pip install pykrx

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from pykrx import stock
print(dir(stock))

['DataFrame', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'datetime', 'deprecated', 'dispatch', 'functools', 'future_api', 'get_business_days', 'get_elw_ticker_list', 'get_elw_ticker_name', 'get_etf_isin', 'get_etf_ohlcv_by_date', 'get_etf_ohlcv_by_ticker', 'get_etf_portfolio_deposit_file', 'get_etf_price_change_by_ticker', 'get_etf_price_deviation', 'get_etf_ticker_list', 'get_etf_ticker_name', 'get_etf_tracking_error', 'get_etf_trading_volume_and_value', 'get_etn_ticker_list', 'get_etn_ticker_name', 'get_etx_ticker_list', 'get_exhaustion_rates_of_foreign_investment', 'get_exhaustion_rates_of_foreign_investment_by_date', 'get_exhaustion_rates_of_foreign_investment_by_ticker', 'get_future_ohlcv', 'get_future_ohlcv_by_ticker', 'get_future_ticker_list', 'get_future_ticker_name', 'get_index_fundamental', 'get_index_fundamental_by_date', 'get_index_fundamental_by_ticker', 'get_index_listing_date', 'get_index_ohlcv', 

In [5]:
# 1. KOSPI 종목 리스트 조회
df_kospi = stock.get_market_ticker_list(market="KOSPI")
print("KOSPI 종목 수:", len(df_kospi))
print(df_kospi[:5])  # 상위 5개 종목 코드

KOSPI 종목 수: 962
['095570', '006840', '027410', '282330', '138930']


In [30]:
import os
import json
from openai import OpenAI

client = OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")

In [40]:
def get_stock_analysis(ticker : str, start_date="20240101", end_date="20240722"):
    df = stock.get_market_ohlcv_by_date(start_date, end_date, ticker)

    closing_prices = df["종가"]
    stock_info = {
        "ticker": ticker,
        "start_date": start_date,
        "end_date": end_date,
        "latest_price": int(closing_prices.iloc[-1]),
        "max_price": int(closing_prices.max()),
        "min_price": int(closing_prices.min()),
        "avg_price": round(closing_prices.mean(), 2),
    }

    return json.dumps(stock_info, ensure_ascii=False)

messages = [{"role": "user", "content": "에이피알 주가를 2024년 1월부터 7월까지 분석해줘."}]
functions = [
    {
        "name": "get_stock_analysis",
        "description": "특정 종목의 주가 데이터를 분석하여 최대, 최소, 평균 종가를 제공합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "종목 코드. 예: 삼성전자 = 005930",
                },
                "start_date": {
                    "type": "string",
                    "description": "조회 시작일 (YYYYMMDD 형식)",
                },
                "end_date": {
                    "type": "string",
                    "description": "조회 종료일 (YYYYMMDD 형식)",
                },
            },
            "required": ["ticker", "start_date", "end_date"],
        },
    }
]

In [41]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    functions=functions,
    function_call="auto",
    )
response_message = response.choices[0].message
print(response_message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"ticker":"261780","start_date":"20240101","end_date":"20240731"}', name='get_stock_analysis'), tool_calls=None)


In [42]:
if response_message.function_call is not None:
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_stock_analysis": get_stock_analysis,
    }
    function_name = response_message.function_call.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(
        ticker=function_args['ticker'],
        start_date=function_args['start_date'],
        end_date=function_args['end_date']
    )

    messages.append(response_message)
    messages.append(
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }
    )
    second_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )  # get a new response from GPT where it can see the function response

    print(second_response.choices[0].message.content)

에이피알(261780)의 2024년 1월부터 7월까지 주가 분석 결과는 다음과 같습니다.

- **최신 주가**: 4,505원
- **최고 주가**: 5,710원
- **최저 주가**: 4,490원
- **평균 주가**: 4,929.97원

이 분석은 해당 기간 동안의 주가 변동을 반영하고 있으며, 현재 주가는 지난 최고가에 비해 낮은 상황입니다. 평균 주가 또한 이를 고려할 때 안정적인 흐름을 보이고 있음을 나타냅니다.
